In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import utils

%matplotlib inline

In [2]:
tracks = utils.load('C:/Users/night/Documents/09/school/actual-masters/git/masters/models/data/fma/fma_metadata/tracks.csv')
genres_ = utils.load('C:/Users/night/Documents/09/school/actual-masters/git/masters/models/data/fma/fma_metadata/genres.csv')

tracks.shape, genres_.shape

((106574, 52), (163, 4))

In [3]:
subset = tracks.index[tracks['set', 'subset'] <= 'medium']
tracks = tracks.loc[subset]

train = tracks.index[tracks['set', 'split'] == 'training']
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']

top_lvl_genres = list(genres_.index[genres_['parent'] == 0])
print(top_lvl_genres)

[2, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15, 17, 20, 21, 38, 1235]


In [7]:
# just getting the top genre, not one hot encoded

# top_lvl_genres = list(genres_.index[genres_['parent'] == 0])
print(top_lvl_genres)
top_lvl_names = []

genres_index = genres_
# genres_index['genre_id'] = genres_index.index
genres_index = genres_index.reset_index()
# genres_index.head()

# print(type(genres_))

print(len(genres_index))

for i, row in enumerate(genres_index):
    if genres_index.iloc[i]['genre_id'] in top_lvl_genres:
        value = genres_index.iloc[i]['genre_id']
        print(value)
        
for j in top_lvl_genres:
    for k in range(163):
        if genres_index.iloc[k]['genre_id'] == j:
            top_lvl_names.append(genres_index.iloc[k]['title'])
        
# genres_index.head(20)
print(top_lvl_names)

[2, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15, 17, 20, 21, 38, 1235]
163
2
3
4
5
['International', 'Blues', 'Jazz', 'Classical', 'Old-Time / Historic', 'Country', 'Pop', 'Rock', 'Easy Listening', 'Soul-RnB', 'Electronic', 'Folk', 'Spoken', 'Hip-Hop', 'Experimental', 'Instrumental']


In [8]:
print(len(tracks))

25000


In [ ]:
columns = ['track_id'] + top_lvl_genres

data = pd.DataFrame(columns=columns)
tracks_df = pd.DataFrame(tracks)
data.head()

# print(tracks_df.columns)

genres_set = set(top_lvl_genres)

for i, track_id in enumerate(subset):
    if i < 100:
        genres = tracks_df.loc[track_id,('track', 'genres_all')]
        if genres_set.intersection(set(genres)) != set():
            data.loc[len(data)] = 0
            data.at[i, 'track_id'] = track_id
            data.at[i, 'split'] = tracks_df.loc[track_id,('set', 'split')]

            genres = tracks_df.loc[track_id,('track', 'genres_all')]
            for genre in genres:
                if genre in top_lvl_genres:
                    data.at[i, genre] = 1
            print(track_id, ':', genres)
        else:
            print('no parent level genres for track', track_id)
        
data.head()
            
# data.set_index('track_id', inplace=True, drop=True)
# data.to_csv('C:/Users/night/Documents/09/school/actual-masters/git/masters/models/data/fma/medium_multigenre.csv')   

In [ ]:
# data.set_index('track_id', inplace=True, drop=True)
data.head()

data.to_csv('C:/Users/night/Documents/09/school/actual-masters/git/masters/models/data/fma/medium_multigenre.csv') 

In [ ]:
total = 0
for col in data:
    if col in top_lvl_genres:
        print(col, data[col].sum())
        total += int(data[col].sum())
print(total)

In [ ]:
# tracks_df = pd.DataFrame(tracks)

# print(tracks_df.loc[2,('track', 'genres')])
# tracks_df.head()

genre_ids = genres_.iloc[0]['title']
# print(genre_ids['title'])
print(genre_ids)

In [ ]:
# data.set_index('track_id', inplace=True, drop=True)

# print(data.columns)
# data.head()

data.drop([16, 166], axis=1)

for col in data.columns:
    print(col)
    if type(col) != str:
        try:
            col = int(col)
            genre_name = genres_.iloc[col]['title']
            data.rename(columns={col:genre_name})
        except:
            print('passed')
        
data.head()

In [ ]:
data[16].sum()

In [ ]:
top_genre = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
print('Top genres ({}): {}'.format(len(top_genre), top_genre))

top_genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres']).classes_)
print('All top genres ({}): {}'.format(len(top_genres), top_genres))

genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres_all']).classes_)
print('All genres ({}): {}'.format(len(genres), genres))

In [ ]:
list1 = set([58, 12, 45])
list2 = set([1, 38])

if list1.intersection(list2) == set():
    print('yes')